# === Design ===

## 146. LRU Cache
[146. LRU Cache](https://leetcode.com/problems/lru-cache/)

## 1396. Design Underground System
[1396. Design Underground System](https://leetcode.com/problems/design-underground-system/)

In [6]:
from collections import defaultdict
a = defaultdict(lambda: [0, 0])
a['cc'] = [1,1]
print(a)

defaultdict(<function <lambda> at 0x7f849de09670>, {'cc': [1, 1]})


In [8]:
from collections import defaultdict

class UndergroundSystem:

    def __init__(self):
        # P
        self.check_in_dict = {}
        # S chose 2 = S^2
        self.trip_dict = defaultdict(lambda: [0, 0])

    def checkIn(self, id: int, stationName: str, t: int) -> None:
        self.check_in_dict[id] = (stationName, t)
        

    def checkOut(self, id: int, stationName: str, t: int) -> None:
        # look up the id in check_in_dict and transfer the data to trip_dict
        start_station, start_time = self.check_in_dict.pop(id)
        self.trip_dict[(start_station, stationName)][0] += t - start_time
        self.trip_dict[(start_station, stationName)][1] += 1
        

    def getAverageTime(self, startStation: str, endStation: str) -> float:
        total_time, total_count = self.trip_dict[(startStation, endStation)]
        return total_time / total_count


# Your UndergroundSystem object will be instantiated and called as such:
# obj = UndergroundSystem()
# obj.checkIn(id,stationName,t)
# obj.checkOut(id,stationName,t)
# param_3 = obj.getAverageTime(startStation,endStation)

## 1656. Design an Ordered Stream
[1656. Design an Ordered Stream](https://leetcode.com/problems/design-an-ordered-stream/)

In [2]:
"""
Basically , we need to store every incoming value at the given index. And
with every incoming index, we have to check

If the current index is less than the incoming index, the we have to return
an empty list

Else , we have to return an sliced list from the incoming index to the first index
where there is no insertion till yet.

Solution:
    Initialize a list of size n with None
    Maintain the current index with self.ptr
    For every insert call, with idKey, value
    Assign the list[idKey-1] to the value # Since array is 0-index reduce 1
    Check if the current index is less than incoming index(idKey-1) and return []
    Else return sliced list from incoming index(idKey-1) till we do not encounter None.
"""

class OrderedStream:
    def __init__(self, n: int):
        self.stream = [None for _ in range(n)]
        self.ptr = 0
        
    def insert(self, idKey: int, value: str):
        idKey -= 1
        self.stream[idKey] = value
        if self.ptr < idKey:
            return []
        else:
            while self.ptr < len(self.stream) and self.stream[self.ptr] is not None:
                self.ptr += 1
            return self.stream[idKey:self.ptr]

# Your OrderedStream object will be instantiated and called as such:
# obj = OrderedStream(n)
# param_1 = obj.insert(idKey,value)

## 380. Insert Delete GetRandom O(1)
[380. Insert Delete GetRandom O(1)](https://leetcode.com/problems/insert-delete-getrandom-o1/)

In [14]:
from random import choice
a = [1,2,3]
b = choice(a)
b

3

In [16]:
from random import choice
class RandomizedSet:

    def __init__(self):
        self.rand_list = []
        self.ele_idx = {}

    def insert(self, val: int) -> bool:
        if val in self.ele_idx:
            return False
        self.ele_idx[val] = len(self.rand_list)
        self.rand_list.append(val)
        return True

    def remove(self, val: int) -> bool:
        if val not in self.ele_idx:
            return False
        
        # find the last element
        last_ele = self.rand_list[-1]
        # find the idx of delete ele
        del_idx = self.ele_idx[val]
        
        # dict
        # update the last_ele
        self.ele_idx[last_ele] = del_idx
        # delete val
        self.ele_idx.pop(val)
        
        # list
        # self.rand_list[-1], self.rand_list[del_idx] = self.rand_list[del_idx], self.rand_list[-1]
        self.rand_list[del_idx] = last_ele
        self.rand_list.pop()
        return True

    def getRandom(self) -> int:
        return choice(self.rand_list)

## 381. Insert Delete GetRandom O(1) - Duplicates allowed
[381. Insert Delete GetRandom O(1) - Duplicates allowed](https://leetcode.com/problems/insert-delete-getrandom-o1-duplicates-allowed/)

# === Array/String ===

## 1. Two Sum
[1. Two Sum](https://leetcode.com/problems/two-sum/)
- 变种 ： two sum变形，find how many pairs‍‌‍‌‌‍‌‍‍‍‌‍‍‍‌‌‌‌‌ (of 2 elements) in an array sum up to the target value，

## 394. Decode String
[394. Decode String](https://leetcode.com/problems/decode-string/)

## 1790. Check if One String Swap Can Make Strings Equal
[1790. Check if One String Swap Can Make Strings Equal](https://leetcode.com/problems/check-if-one-string-swap-can-make-strings-equal/)

- 判断两个string，如果 只做一次swap ，是否可以变的一样

## 611. Valid Triangle Number
[611. Valid Triangle Number](https://www.1point3acres.com/bbs/thread-918542-1-1.html)

## 723. Candy Crush
[723. Candy Crush](https://leetcode.com/problems/candy-crush/)

## ++++  Intervals +++

## 253. Meeting Rooms II
[253. Meeting Rooms II](https://leetcode.com/problems/meeting-rooms-ii/)

In [2]:
class Solution:
    def minMeetingRooms(self, intervals) -> int: 
        """
        Minimum number of conference room --- maximium number of overlapping at the same time. 
        [[0,30], [5,10], [15,20]] 
        
        time stamp: 
        start time: +1 
        end time: -1 
        
        list: (time,number)
        0 +1; 30 -1; 5: +1; etc. 
        
        sort() 
        
        0   5    10    15    20    30 
        1   1     -1    1     -1    -1   
        
        arr = 1   2      1     2     1    0       
        
        max(arr) 
        time: n log (n) 
        space: O(n)  
        """   
        # dict time stamps
        time_stamps = []
        for start, end in intervals:
            # cover smae time stamps
            time_stamps.append((start, 1))
            time_stamps.append((end, -1))

        # sort
        time_stamps.sort()

        # calculate the accumulated sum and keep track of the global max
        global_max = 0
        cur_room = 0
        for time, val in time_stamps:
            cur_room += val
            global_max = max(global_max, cur_room)
        # return global max
        return global_max

## 759. Employee Free Time
[759. Employee Free Time](https://leetcode.com/problems/employee-free-time/)

简单来说就是 给定一个总体的time frame，然后再给你一个time interval的list 让你找出其中空出来的interval
被挂倒是不冤 第一轮我就给出了灾难级的表现 国人大哥给接雨水那题很可能是想帮我 毕竟算是刷题网很经典的题了 可惜我没把握住 上一次做这题得半年前了
我就给了一个当时自己做的解法 然后让我优化 我依稀记得可以用单调栈 但是中间很多的细节记不起来

In [4]:
# optmize by default dict (avoid same time stamps has not interval)
class Solution:
    def employeeFreeTime(self, schedule: '[[Interval]]') -> '[Interval]':
        """
        [[1,2],   [5,6]],
        [[1,  3]],
               [[4,       10]]
         2 -1 -1 1 1  -1   -1
         2  1  0 1 2  1     0
        
        accumulated sum == 0
        
        
        edge case:
                       [94,99]
        [57,87],[91,94]
        1    -1  1  -1
                         1  -1
        1    0   1   0   1   0
        """
        # time_stamps
        time_stamps = defaultdict(int)
        for cur in schedule:
            for s in cur:
                time_stamps[s.start] += 1
                time_stamps[s.end] -= 1
        # sort
        time_sort = sorted(time_stamps.keys())
        # accumulated sum
        cur_sum = 0
        res = []
        for i in range(len(time_sort) - 1):
            pre = time_sort[i]
            val = time_stamps[pre]
            cur_sum += val
            if cur_sum == 0:
                cur = time_sort[i + 1]
                res.append(Interval(pre, cur))
        return res

In [3]:
class Solution:
    def employeeFreeTime(self, schedule: '[[Interval]]') -> '[Interval]':
        """
        [[1,2],   [5,6]],
        [[1,  3]],
               [[4,       10]]
         2 -1 -1 1 1  -1   -1
         2  1  0 1 2  1     0
        
        accumulated sum == 0
        
        
        !! edge case:
                       [94,99]
        [57,87],[91,94]
        1    -1  1  -1
                         1  -1
        1    0   1   0   1   0
        """
        # time_stamps
        time_stamps = []
        for cur in schedule:
            for s in cur:
                time_stamps.append((s.start, 1))
                time_stamps.append((s.end, -1))
        # sort
        time_stamps.sort()
        # accumulated sum
        accumulated_sum = []
        cur_sum = 0
        for time, val in time_stamps:
            cur_sum += val
            accumulated_sum.append((time, cur_sum))
        # post processing
        res = []
        for i in range(len(accumulated_sum) - 1):
            pre = accumulated_sum[i]
            cur = accumulated_sum[i + 1]
            if pre[1] == 0 and pre[0] != cur[0]:
            # if pre[1] == 0:
                res.append(Interval(pre[0], cur[0]))
        return res

## 56. Merge Intervals
[56. Merge Intervals](https://leetcode.com/problems/merge-intervals/)

In [8]:
class Solution:
    def merge(self, intervals):
        """
        TC = O(nlogn)
        SC = O(n)
        
        sort by start time
        [1,3],[2,6],[8,10],[15,18]
        compare pair by pair
        case 1: cur start > previous end (not merge)
        case 2: cur start <= previous end (merge)
        """
        res = []
        n = len(intervals)
        intervals.sort()
        res.append(intervals[0])
        for i in range(1, n):
            cur = intervals[i]
            if cur[0] > res[-1][1]:
                # not merge
                res.append(cur)
            else:
                # merge
                res[-1] = [res[-1][0], max(res[-1][1], cur[1])]
        return res

## 986. Interval List Intersections
[986. Interval List Intersections](https://leetcode.com/problems/interval-list-intersections/)

In [9]:
# Solution 2: merge intervals
class Solution:
    def intervalIntersection(self, A, B):
        ans = []
        i = j = 0

        while i < len(A) and j < len(B):
            # Let's check if A[i] intersects B[j].
            # lo - the startpoint of the intersection
            # hi - the endpoint of the intersection
            lo = max(A[i][0], B[j][0])
            hi = min(A[i][1], B[j][1])
            if lo <= hi:
                ans.append([lo, hi])

            # Remove the interval with the smallest endpoint
            if A[i][1] < B[j][1]:
                i += 1
            else:
                j += 1

        return ans

In [6]:
# Solution 1: accummulated sum
# !!! edge case same time stamp one start one end
class Solution:
    def intervalIntersection(self, firstList, secondList):
        """
        [0,  2],  [5,10]
          [1,   5]
         1 1 -1 -1 1 -1
         1 2  1  0 1  0
        
        !!! edge case
        same time stamp: one is start, one is end
        """
        time_stamps = []
        for start, end in firstList:
            time_stamps.append((start, 1))
            time_stamps.append((end, -1))
        for start, end in secondList:
            time_stamps.append((start, 1))
            time_stamps.append((end, -1))
        # sort
        time_stamps.sort()
        # find the intervals with val 2
        cur_sum = 0
        res = []
        for i in range(len(time_stamps) - 1):
            pre = time_stamps[i]
            cur = time_stamps[i + 1]
            cur_sum += pre[1]
            if cur_sum == 2 or (pre[0] == cur[0] and pre[1] + cur[1] == 0):
                res.append((pre[0], cur[0]))
        return res

## 42. Trapping Rain Water
[42. Trapping Rain Water](https://leetcode.com/problems/trapping-rain-water/)

# === Heap / PriorityQueue ===

## 1244. Design A Leaderboard
[1244. Design A Leaderboard](https://leetcode.com/problems/design-a-leaderboard/)

In [3]:
from collections import defaultdict
import heapq
class Leaderboard:
    # SC = O(N + K) dict + heap

    def __init__(self):
        self.scores = defaultdict(int)

    # TC = O(1)
    def addScore(self, playerId: int, score: int) -> None:
        self.scores[playerId] += score
        
    # TC = K (heapify) + (N - K)logK(pop) + KlogK(add up)
    def top(self, K: int) -> int:
        minHeap = []
        for x in self.scores.values():
            heapq.heappush(minHeap, x)
            if len(minHeap) > K:
                heapq.heappop(minHeap)
        # add top k
        total = 0
        while minHeap:
            total += heapq.heappop(minHeap)
        return total
        
    # TC = O(1)
    def reset(self, playerId: int) -> None:
        self.scores[playerId] = 0

# === HashTable ===

## 1347. Minimum Number of Steps to Make Two Strings Anagram
[1347. Minimum Number of Steps to Make Two Strings Anagram](https://leetcode.com/problems/minimum-number-of-steps-to-make-two-strings-anagram/)

In [2]:
class Solution:
    def minSteps(self, s: str, t: str) -> int:
        """
        1. calculate the char an it's count
            s leetcode -> {l:1, e:3, t:1, c:1, o:1, d:1}
            t practice -> {p:1, e:1, r:1, c:2, a:1, t:1, i:1}
        2. iterate over t:
            if ch in s:
                if s_count < t_count: need to modify the dif
                if s_count > t_count: offset the dif
           if ch not in s:
                modify ch
        """
        s_dict = Counter(s)
        t_dict = Counter(t)
        steps = 0
        for ch, count in t_dict.items():
            if ch in s_dict:
                dif = count - s_dict[ch]
                steps += dif if dif > 0 else 0
            else:
                steps += count
        return steps

## 1169. Invalid Transactions
[1169. Invalid Transactions](https://leetcode.com/problems/invalid-transactions/)

In [1]:
from collections import defaultdict
"""
Clarify:
    1. duplicate output
    
Solution:
        prev_tran = dictionary --> create a dictionary to keep track of previous transaction 
        invalid_tran = set --> to store invalid transaction /set to avoid duplication 
        
        Iterate over transactions:
            -> if the amount is greater than 1000 add it to the invalid_tran
            -> if there is any other previous transaction done by similar person , check it from the prev_tran
                    -> bring all previous transaction done by similar person (iterate over the prev_tran)
                            -> check if the absolute time difference is less than 60 and the city is different 
                            -> if that is true add it to the invalid transaction 
            -> add the transaction to the prev_tran - always keep track of previous transaction 
"""
class Solution:
    def invalidTransactions(self, transactions):
        if not transactions:
            return []
        prev_tran = defaultdict(list)
        invalid_tran = set()
        for i, tran in enumerate(transactions):
            cur_info = tran.split(',')
            # get info
            name, time, amount, city = cur_info[0], int(cur_info[1]), int(cur_info[2]), cur_info[3]
            # check amount
            if amount > 1000:
                invalid_tran.add((tran, i))
            # check same person transactions
            if name in prev_tran:
                for pre, idx in prev_tran[name]:
                    prev_info = pre.split(',')
                    prev_time = int(prev_info[1])
                    prev_city = prev_info[3]
                    if abs(prev_time - time) <= 60 and prev_city != city:
                        invalid_tran.add((pre,idx))
                        invalid_tran.add((tran, i))
            # add current transaction into pre_tran
            prev_tran[name].append((tran, i))
        return [ele[0] for ele in invalid_tran]
# TC = O(n^2)
# SC = O(n)

# === Linkedlist ===

## 83. Remove Duplicates from Sorted List
[83. Remove Duplicates from Sorted List](https://leetcode.com/problems/remove-duplicates-from-sorted-list/)

## 24. Swap Nodes in Pairs
[24. Swap Nodes in Pairs](https://leetcode.com/problems/swap-nodes-in-pairs/)

## 2. Add Two Numbers
[2. Add Two Numbers](https://leetcode.com/problems/add-two-numbers/)

## 445. Add Two Numbers II
[445. Add Two Numbers II](https://leetcode.com/problems/add-two-numbers-ii/)

## 430. Flatten a Multilevel Doubly Linked List
[430. Flatten a Multilevel Doubly Linked List](https://leetcode.com/problems/flatten-a-multilevel-doubly-linked-list/)

In [3]:
"""
# Definition for a Node.
class Node:
    def __init__(self, val, prev, next, child):
        self.val = val
        self.prev = prev
        self.next = next
        self.child = child
"""

class Solution:
    """
    TC = O(n)
    SC = O(n)
    
    1. iterate over the current level
    2. if cur node has child, recursion(keep track if the head and taill of subproblem)
    3. connect sub_problem node to cuurent level node
        
    """
    def flatten(self, head: 'Optional[Node]') -> 'Optional[Node]':
        return self.helper(head)[0]
    
    def helper(self, head):
        # base case
        if not head:
            return head, head
        # current level
        cur = head
        while cur:
            if not cur.child:
                cur = cur.next
            else:
                child_head, child_tail = self.helper(cur.child)
                # record next node
                next_node = cur.next
                if cur.next:
                    child_tail.next = next_node
                    next_node.prev = child_tail
                    
                cur.next = child_head
                child_head.prev = cur
                cur.child = None
                # next loop
                cur = next_node
        # record tail of current list
        tail = head
        while tail.next:
            tail = tail.next
        return head, tail

# === Queue/Stack/Deque ===

## 496. Next Greater Element I
[496. Next Greater Element I](https://leetcode.com/problems/next-greater-element-i/)

## 895. Maximum Frequency Stack
[895. Maximum Frequency Stack](https://leetcode.com/problems/maximum-frequency-stack/)

## 1472. Design Browser History
[1472. Design Browser History](https://leetcode.com/problems/design-browser-history/)

In [4]:
class BrowserHistory:
    """
    two stack to keep track of the dependency of previous and next pages
    
    homepage: b
    stack previous [ a 
    stack next [ c
    
    # !!! when visiting a new page, no next url exist
    """

    def __init__(self, homepage: str):
        self.home = homepage
        self.previous = deque()
        self.next = deque()

    def visit(self, url: str) -> None:
        self.previous.append(self.home)
        self.home = url
        # !!! when visiting a new page, no next url exist
        self.next.clear()

    def back(self, steps: int) -> str:
        while self.previous and steps > 0:
            self.next.append(self.home)
            self.home = self.previous.pop()
            steps -= 1
        return self.home

    def forward(self, steps: int) -> str:
        while self.next and steps > 0:
            self.previous.append(self.home)
            self.home = self.next.pop()
            steps -= 1
        return self.home

# === Greedy ===

## 1029. Two City Scheduling
[1029. Two City Scheduling](https://leetcode.com/problems/two-city-scheduling/)

In [18]:
# TC = O(nlogn)
# SC = O(n)
class Solution:
    def twoCitySchedCost(self, costs) -> int:
        """
        diff = [B - A] (diff is larger, the cost to B is larger)
        sort diff
        put first half people to A, and second half to B
        """
        diffs = []
        for c1, c2 in costs:
            diffs.append([c2 - c1, c1, c2])
        diffs.sort()
        res = 0
        for i in range(len(diffs)):
            if i < len(diffs) // 2:
                res += diffs[i][2]
            else:
                res += diffs[i][1]
        return res

# === Tree ===

## 98. Validate Binary Search Tree
[98. Validate Binary Search Tree](https://leetcode.com/problems/validate-binary-search-tree/)
- O(1) space

## 314. Binary Tree Vertical Order Traversal
[314. Binary Tree Vertical Order Traversal](https://leetcode.com/problems/binary-tree-vertical-order-traversal/)

## 428. Serialize and Deserialize N-ary Tree
[428. Serialize and Deserialize N-ary Tree](https://leetcode.com/problems/serialize-and-deserialize-n-ary-tree/)

## === DFS ===

## 200. Number of Islands
[200. Number of Islands](https://leetcode.com/problems/number-of-islands/)

## 694. Number of Distinct Islands
[694. Number of Distinct Islands](https://leetcode.com/problems/number-of-distinct-islands/)

## 46. Permutations
[46. Permutations](https://leetcode.com/problems/permutations/)

## 47. Permutations II
[47. Permutations II](https://leetcode.com/problems/permutations-ii/)

## 1274. Number of Ships in a Rectangle
[1274. Number of Ships in a Rectangle](https://leetcode.com/problems/number-of-ships-in-a-rectangle/)

## === DP ===

## 140. Word Break II
[140. Word Break II](https://leetcode.com/problems/word-break-ii/)

## 935. Knight Dialer
[935. Knight Dialer](https://leetcode.com/problems/knight-dialer/)

https://www.1point3acres.com/bbs/thread-916935-1-1.html
8.2 - 8.14